# **Project: Solar Cycle**

**Course:** Optimisation and High performance Computing (OHPC-HS25-AD23)  
**Team Members:** Cieplinski Nicole, Plos Penelope, Yeji Huber
**Date:** 16.01.2026

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
from pathlib import Path
import glob

## Simulated Annealing (SA) function

### SA function for hyper-parameter tuning

In [ ]:
def sa_tune(x0, T0, sigma, f, n_iter = 2e5, thinning = 1):

    x = x0.copy()           # initial x
    T = T0                  # initial temperature
    n_params = x0.shape[0]  # number of parameters to optimized

    # Means and covariance matrix for the jump distribution -> multivariate normal with mean 0 and standard deviation sigma 
    means = np.full(n_params, 0)
    cov_matrix = np.diag(np.full(n_params, sigma))

    # Calculate size of the output array after thinning
    # (thinning -> save states at regular intervals instead of every iteration) 
    # Thinning is by defaut 1, and size_out = n_iter
    size_out = int((n_iter + thinning -1)//thinning)    # equivalent to ceiling (n_iter/thinning)
    v = np.zeros((size_out, n_params))
    # Store the initial parameter array
    v[0,:] = x

    iter_counter = 0
    iter_counter_thin = 0
    print("Initial loss:", f(x))
    #start main loop
    while iter_counter < n_iter:
        iter_counter += 1;
        x_old = x;
        x_proposal = x_old + np.random.multivariate_normal(means, cov_matrix)
        DeltaE = f(x_proposal) - f(x_old)
        #Metropolis accept/reject step 
        if np.exp(-np.clip(DeltaE, -100, 100)) >= np.random.rand():
            x = x_proposal
        else:
            x = x_old
        
        # Update temperature according to schedule
        T = T0 * (1 - iter_counter/n_iter)
        # Keep track of accepted state
        if iter_counter%1 == 0:
            print("Iteration", iter_counter, " - Temperature:", T, "Loss", f(x))
        if iter_counter%thinning == 0:
            v[iter_counter_thin,:] = x
            iter_counter_thin += 1
    
    return v
    

### SA function for final optimization

In [ ]:
def sa_optimize(x0, T0, sigma, f, n_iter = 2.5e5, burn_in = 2e5):

    x = x0.copy()           # initial x
    T = T0                  # initial temperature
    n_params = x0.shape[0]  # number of parameters to optimized
    
    # means and covariance matrix for the jump distribution -> multivariate normal
    means = np.full(n_params, 0)
    cov_matrix = np.diag(np.full(n_params, sigma))

    # Size of the output array after burn_in
    size_out = int(n_iter - burn_in)
    v = np.zeros((size_out, n_params))
    
    iter_counter = 0
    print("Initial loss:", f(x))
    # Start main loop
    while iter_counter < n_iter:
        iter_counter += 1;
        x_old = x;
        x_proposal = x_old + np.random.multivariate_normal(means, cov_matrix)
        DeltaE = f(x_proposal) - f(x_old)
        # Metropolis accept/reject step
        if np.exp(-np.clip(DeltaE/T, -100,100)) >= np.random.rand():
            x = x_proposal
        else:
            x = x_old
        # Update temperature according to schedule
        T = T0*(1-iter_counter/n_iter)
        # keep track of the algorithm state
        if iter_counter%10 == 0:
            print("Iteration ", iter_counter, " - Temperature:", T, " - Loss:", f(x))
        if iter_counter > burn_in:
            v[iter_counter-int(burn_in)-1, :] = x

    return v 

## Load data

In [ ]:
data = np.loadtxt('data_Team9.csv', delimiter=',', skiprows=1) 

In [ ]:
data.shape

In [ ]:
data

In [ ]:
#Plot data for visualization
plt.figure()
plt.scatter(data[:,0], data[:,1], color="Orange", s=10, label= "Data")
plt.xlabel("Time(Day)")
plt.ylabel("SN")
plt.title("Solar Cycle")
plt.legend()
plt.show()

In [ ]:
time_points = data[:, 0]
n_data = time_points.shape[0]
data_points = data[:, 1]

print(time_points)
print(n_data)
print(data_points)

## Initial conditions

### parameters              
 Phase 1: T01, Ts1, Td1     
 Phase 2: T02, Ts2, Td2         
...     
 Phase 10: T010, Ts10, Td10   

### Ts and Td

In [ ]:
# x0 = np.array([Ts1, Td1, Ts2, Td2, ...])
# here we use initial conditions given in the final project intro 

x0 = np.array([0.3, 5, 0.3, 5, 0.3, 5, 0.3, 5, 0.3, 5, 0.3, 5, 0.3, 5, 0.3, 5, 0.3, 5, 0.3, 5])

In [ ]:
# The different parameters can be extracted as follows:
Ts = x0[::2]
Td = x0[1::2]

print(Ts)
print(Td)

# Number of phases
num_phases = len(Ts)
num_phases

### T0

In [ ]:
# make T0 array with initial conditions reported in Hathaway 2015
T0array = (1878.916666666666667, 1890.166666666666667, 1902.000000000000000, 1913.500000000000000, 1923.583333333333333, 1933.666666666666667, 1944.083333333333333, 1954.250000000000000, 1964.750000000000000, 1976.166666666666667, 1986.666666666666667)
# Create list of time intervals for each phase
intervals = [(float(T0array[ix]),float(T0array[ix+1])) for ix in range(num_phases)]

print(intervals)

In [ ]:
# We will use a loop to process all phases
for ix, (a, b) in enumerate(intervals):
    print("Processing phase", ix+1, "with interval (", a, ",", b, ")")

## Model and Loss Function

### Model

In [ ]:
# Model for multiple phases:
def model(t, x):
    Ts = x[::2]
    Td = x[1::2]
    num_phases = len(T0array)
    intervals = [(T0array[ix],T0array[ix+1]) for ix in range(num_phases-1)]
    #Ensure t is treated as an array for consistency
    t = np.atleast_1d(t)
    model_output = np.zeros_like(t)
    for ix, (a,b) in enumerate(intervals):
        # Create mask for current phase
        mask = (a <= t) & (t < b)
        # Apply model for current phase
        model_output[mask] = ((t[mask] - T0array[ix])/Ts[ix])**2 * np.exp(-((t[mask] - T0array[ix])/Td[ix])**2)
    if model_output.size == 1:
        return model_output.item()
    else:
        return model_output


### Loss function

In [ ]:
# Loss function
def mse(x):
    return np.mean(np.square(data_points - model(time_points, x)))

In [ ]:
# Visualisation
plt.figure()
plt.scatter(time_points, data_points, color='orange', s=10)
plt.plot(time_points, model(time_points, x0), color='blue', linestyle='--', linewidth=1)
plt.show(block=False)

# Initial mse:
mse(x0)

## Hyper-parameters tuning

#### Quick exploration (showing our process)
Before running a full sweep, we tested a few sigma and T0 values to find a reasonable range (avoiding unstable jumps and ensuring the loss decreases).
We log the final MSE for each trial.

In [ ]:
trial_log = []  # list of dicts

def log_trial(T0, sigma, final_x):
    trial_log.append({
        "T0": float(T0),
        "sigma": float(sigma),
        "final_mse": float(mse(final_x))
    })

### Test to find the sigma range

In [ ]:
T0 = 1
sigma = 1
outSA = sa_tune(x0, T0, sigma, mse, 1e4, thinning = 1)

plt.figure()
mse_curve = np.apply_along_axis(mse, 1, outSA)
plt.plot(mse_curve)
plt.show(block = False)

mse(outSA[-1])
log_trial(T0, sigma, outSA[-1])

sigma works

In [ ]:
T0 = 0.1
sigma = 0.00001
outSA = sa_tune(x0, T0, sigma, mse, 1e4, thinning = 1)

plt.figure()
mse_curve = np.apply_along_axis(mse, axis=1, arr=outSA)
plt.plot(mse_curve)
plt.xlabel("Iteration")
plt.ylabel("MSE")
plt.show(block=False)

log_trial(T0, sigma, outSA[-1])
mse(outSA[-1])

In [ ]:
df_trials = pd.DataFrame(trial_log).sort_values("final_mse").reset_index(drop=True)
df_trials.head(10)

## Hyperparameter tuning on the cluster (8×8 = 64)
Load the 64 JSON outputs from the Slurm job array and pick the best (T0, sigma).

In [ ]:
tuning_dir = Path("results_tuning")
tuning_files = sorted(tuning_dir.glob("tuning_*.json"))

assert len(tuning_files) == 64, f"Expected 64 tuning files, found {len(tuning_files)}"

rows = []
for path in tuning_files:
    with path.open("r") as f:
        rows.append(json.load(f))

df_tune = pd.DataFrame(rows).sort_values("final_mse").reset_index(drop=True)
df_tune[["idx", "T0", "sigma", "final_mse", "wall_time_sec"]].head(10)

In [ ]:
best = df_tune.iloc[0]
T0_opt = best["T0"]
sigma_opt = best["sigma"]
x_opt_from_sweep = np.array(best["final_x"], dtype=float)

print("Best hyperparameters from 64-run sweep:")
print("T0_opt =", T0_opt)
print("sigma_opt =", sigma_opt)
print("best MSE =", best["final_mse"])

#### Visual check of the best tuned solution
We plot the best tuned model output against the data as a sanity check before final optimization.

In [ ]:
plt.figure()
plt.scatter(time_points, data_points, color='orange', s=20)
plt.plot(time_points, model(time_points, x_opt_from_sweep), color='blue')
plt.show(block=False)

## Calibration on the cluster (parallel chains + speedup)
Load calibration JSON files and extract the fitted parameters (center_of_mass). Plot wall time vs cores.

In [ ]:
calib_dir = Path("results_calibration")
calib_files = sorted(calib_dir.glob("calib_workers*_chains*.json"))

assert len(calib_files) > 0, "No calibration result files found in results_calibration/"

rows = []
for path in calib_files:
    with path.open("r") as f:
        rows.append(json.load(f))

df_calib = pd.DataFrame(rows).sort_values("n_workers").reset_index(drop=True)
df_calib[["n_workers", "wall_time_sec", "final_mse", "n_chains"]]

In [ ]:
plt.figure()
plt.plot(df_calib["n_workers"], df_calib["wall_time_sec"], marker="o")
plt.xlabel("Cores (cpus-per-task)")
plt.ylabel("Wall time (s)")
plt.show()

t1 = float(df_calib[df_calib["n_workers"]==1]["wall_time_sec"].iloc[0])
df_calib["speedup"] = t1 / df_calib["wall_time_sec"]

plt.figure()
plt.plot(df_calib["n_workers"], df_calib["speedup"], marker="o")
plt.xlabel("Cores (cpus-per-task)")
plt.ylabel("Speedup (T1/Tp)")
plt.show()

In [ ]:
row_final = df_calib.sort_values("n_workers").iloc[-1]  # or choose lowest MSE
center_of_mass = np.array(row_final["center_of_mass"], dtype=float)

print("Using calibration from", row_final["n_workers"], "cores")
print("Calibration MSE:", row_final["final_mse"])

### correlation between Ts and Td

In [ ]:
# Your model uses pairs: [Ts1, Td1, Ts2, Td2, ...]
ts = center_of_mass[0::2]
td = center_of_mass[1::2]

slope, intercept = np.polyfit(ts, td, 1)
td_fit = slope * ts + intercept

plt.figure()
plt.scatter(ts, td, s=20)
plt.plot(ts, td_fit, label=f'Fit: td = {slope:.4f} * ts + {intercept:.4f}')
plt.xlabel('Ts')
plt.ylabel('Td')
plt.legend()
plt.show(block=False)

slope, intercept

In [ ]:
plt.figure()
plt.scatter(time_points, data_points, s=10, label="Data")
plt.plot(time_points, model(time_points, center_of_mass), label="Final calibrated model")
plt.legend()
plt.show()

### R^2 coefficient

In [ ]:
residuals = td - td_fit

ss_total = np.sum((td - np.mean(td)) ** 2)
ss_residual = np.sum(residuals ** 2)

r_squared = 1 - (ss_residual / ss_total)
print(f"R² = {r_squared:.3f}")

### comparison to literature 

In [ ]:
# If the literature reports a slope per year vs per day (or vice versa),
# you may need a rough rescaling depending on your time resolution.
# Teacher example divides by 365 for ~daily resolution.

slope_per_year_est = slope / 365.0
print("slope =", slope)
print("slope/365 =", slope_per_year_est)

# Add your literature target value here once you have it:
# print("Literature slope ~", 0.02)  # example placeholder

In [ ]:
pred = model(time_points, center_of_mass)
res = data_points - pred

plt.figure()
plt.scatter(time_points, res, s=10)
plt.axhline(0)
plt.xlabel("Time")
plt.ylabel("Residual (data - model)")
plt.show(block=False)

print("Final MSE =", np.mean(res**2))